In [1]:
import pandas as pd
import numpy as np

fh = open('4lzm.pdb')

position = []
for line in fh:
    if line.startswith('ATOM'):
        position.append(line.strip())
        #print(line)

#print(position)

C_alpha = []
for element in position:
    if 'CA' in element:
        C_alpha.append(element)
        #print(element)

#print(C_alpha,'\n',len(C_alpha))

prot_coord = []
for atom in C_alpha:
    coordinates =[]
    atom = atom.split(' ')
    atom = [item.strip() for item in atom if item.strip()][-6:-3]   #if item.strip(): This checks if the stripped version of item is not an empty string. If item contains only whitespace, 
                                                                    #calling strip() will result in an empty string, which will evaluate to False in the if statement.
    for element in atom:
        if element.isnumeric:
            coordinates.append(element)
    prot_coord.append(coordinates)

print(prot_coord)


[['43.729', '-1.868', '8.977'], ['40.567', '-0.840', '10.911'], ['38.437', '2.279', '11.338'], ['40.662', '3.614', '14.094'], ['43.923', '3.407', '12.185'], ['42.203', '4.776', '9.105'], ['40.872', '7.852', '10.912'], ['44.165', '8.321', '12.730'], ['45.886', '8.615', '9.365'], ['43.328', '11.094', '8.026'], ['43.009', '13.349', '11.074'], ['46.455', '13.147', '12.699'], ['46.905', '13.704', '16.470'], ['47.509', '17.072', '18.139'], ['47.621', '17.618', '21.900'], ['47.057', '21.414', '21.847'], ['43.998', '23.342', '20.506'], ['44.411', '23.887', '16.780'], ['42.412', '25.101', '13.843'], ['41.058', '22.509', '11.413'], ['41.015', '22.870', '7.647'], ['37.840', '25.032', '7.919'], ['39.437', '27.215', '10.563'], ['37.540', '25.716', '13.495'], ['38.901', '25.008', '16.927'], ['39.782', '21.286', '17.325'], ['41.917', '19.063', '19.427'], ['43.168', '15.422', '19.501'], ['42.162', '13.162', '16.629'], ['39.791', '15.534', '14.749'], ['37.509', '16.304', '17.720'], ['35.695', '19.580',

In [2]:
def coordinates(file):
    fh = open(file)

    position = []
    for line in fh:
        if line.startswith('ATOM'):
            position.append(line.strip())
            #print(line)

    C_alpha = []
    for element in position:
        if 'CA' in element:
            C_alpha.append(element)
            #print(element)

    prot_coord = []
    for atom in C_alpha:
        coordinates =[]
        atom = atom.split(' ')
        atom = [item.strip() for item in atom if item.strip()][-6:-3]   #if item.strip(): This checks if the stripped version of item is not an empty string. If item contains only whitespace, 
                                                                        #calling strip() will result in an empty string, which will evaluate to False in the if statement.
        for element in atom:
            if element.isnumeric:
                coordinates.append(float(element))
        prot_coord.append(np.array(coordinates))
    return prot_coord

p4lzm = coordinates('4lzm.pdb')
p6lmz = coordinates('6lzm.pdb')

values = []
for m,n in zip(p4lzm,p6lmz):
    dif_vec = (m - n)
    module = (dif_vec[0]**2+dif_vec[1]**2+dif_vec[2]**2)**(1/2)
    values.append(module)


module_sq = [float(i)**2 for i in values]
print(type(module_sq[0]))
print(len(values))
RMSD = (sum(module_sq)/len(values))**(1/2)
print(RMSD)

<class 'float'>
162
0.16689401160949738


In [48]:
handle = [
    'ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLU', 'GLN', 'GLY', 'HIS', 'ILE',
    'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL'
]

fh = open('alpha_C.txt')

processer = []
for line in fh:
    if line.strip().endswith('C'):
        processer.append(line)

leu = []

for entry in processer:
    if 'LEU' in entry:
        leu.append(entry)

#print(f'The number of leucines in this protein is {len(entry)}, which accounts for {len(entry)*100/len(processer)}% of the aminoacids')

final = len(processer[1].split())

# Create a dictionary with the appropriate entries
df_dict = dict()

columns_title = [
                'ATOM',
                'atom position',
                'Type',
                'Aminoacid',
                'A',
                'aa position',
                'x',
                'y',
                'z',
                '1',
                'number',
                'Atom']

for n in columns_title:
    df_dict[n] = []
print(df_dict)
#Assing each value from the processed file into an entry

for entry in processer:
    sep = entry.split()
    for x,y in zip(range(0,final),columns_title):
        df_dict[y].append(sep[x])

df = pd.DataFrame(df_dict)

newdf = df.loc[:,['x','y','z']]

def vector_module(row):
    module = ((float(row['x'])**2+float(row['y'])**2+float(row['z'])**2)**(1/2))
    return module

def vector_array(row):
    vector = np.array([row[n] for n in ['x','y','z']])
    return vector

newdf['Module'] = newdf.apply(vector_module,axis=1)
newdf['Vectors'] = newdf.apply(vector_array, axis=1)

aa_list = df['Aminoacid'].tolist()
print(len(aa_list))
counts = dict()
for aminoacid in handle:
    for entry in aa_list:
        if aminoacid in entry:
            if aminoacid not in counts:
                counts[aminoacid] = 1
            
            if aminoacid in counts:
                counts[aminoacid] += 1

print(counts)

print((counts.values()))

{'ATOM': [], 'atom position': [], 'Type': [], 'Aminoacid': [], 'A': [], 'aa position': [], 'x': [], 'y': [], 'z': [], '1': [], 'number': [], 'Atom': []}
162
{'ALA': 16, 'ARG': 14, 'ASN': 12, 'ASP': 11, 'CYS': 3, 'GLU': 9, 'GLN': 6, 'GLY': 12, 'HIS': 2, 'ILE': 11, 'LEU': 16, 'LYS': 14, 'MET': 6, 'PHE': 6, 'PRO': 4, 'SER': 7, 'THR': 12, 'TRP': 4, 'TYR': 7, 'VAL': 10}
dict_values([16, 14, 12, 11, 3, 9, 6, 12, 2, 11, 16, 14, 6, 6, 4, 7, 12, 4, 7, 10])


## Hidden Markov Models

In [8]:
from hmmlearn import hmm

# Example data (replace with your own data)
sequences = [[0, 1, 0, 1, 0], [1, 0, 1, 0, 1], [0, 0, 1, 1, 0]]

# Create an HMM instance
model = hmm.MultinomialHMM(n_components=2)  # You can adjust the number of states

# Fit the model to the data
model.fit(sequences)

# Predict the most likely state sequence for a new observation sequence
new_sequence = [[0, 1, 0, 0, 1]]
predicted_states = model.predict(new_sequence)
print("Predicted states:", predicted_states)

decoded_states = model.decode(new_sequence)
print("Decoded states:", decoded_states)



MultinomialHMM has undergone major changes. The previous version was implementing a CategoricalHMM (a special case of MultinomialHMM). This new implementation follows the standard definition for a Multinomial distribution (e.g. as in https://en.wikipedia.org/wiki/Multinomial_distribution). See these issues for details:
https://github.com/hmmlearn/hmmlearn/issues/335
https://github.com/hmmlearn/hmmlearn/issues/340


Predicted states: [0]
Decoded states: (-209.56517535606667, array([0], dtype=int64))
